<a href="https://colab.research.google.com/github/reachrkr/llamaindexrag/blob/main/chroma_store_example_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q optimum[exporters]
!pip install -q pypdf

In [ ]:
#!pip install -q llama-index
#!pip install -q openai
#!pip install -q transformers==4.31.0
#!pip install -q accelerate
#!pip install -q optimum[exporters]
#!pip install -q InstructorEmbedding
#!pip install -q sentence_transformers
#!pip install -q pypdf
#!pip install -q chromadb

#!pip install -qU \
#  transformers==4.31.0 \
#  sentence-transformers==2.2.2 \
#  accelerate==0.21.0 \
#  langchain==0.0.303 \
#  xformers==0.0.20 \
#  bitsandbytes==0.41.0 \
#  ipywidgets \
#  tiktoken \
#  pydantic==1.10.11
#  chromadb==0.4.15
#!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off
!pip install auto-gptq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.


In [ ]:
!pip install -qU \
  ipywidgets \
  tiktoken \
  pydantic==1.10.11

In [2]:
#!pip install -q llama-index chromadb --quiet
#!pip install -q chromadb
#!pip install -q sentence-transformers
#!pip install -q pydantic==1.10.11

In [3]:
#from google.colab import userdata
#openapi_key=userdata.get('OPENAI_API_KEY')

In [4]:
import os
#os.environ["OPENAI_API_KEY"] = openapi_key

In [ ]:
# !curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf


! curl https://arxiv.org/pdf/2106.07178.pdf --output AD1.pdf
! curl https://arxiv.org/pdf/1404.4679.pdf --output AD2.pdf
! curl https://www.kdd.org/exploration_files/18-1-Article1.pdf --output AD3.pdf

In [6]:
! ls -ltr

total 2292
drwxr-xr-x 1 root root    4096 Nov  9 14:25 sample_data
-rw-r--r-- 1 root root    1923 Nov 11 10:43 AD1.pdf
-rw-r--r-- 1 root root    1922 Nov 11 10:43 AD2.pdf
-rw-r--r-- 1 root root 2331363 Nov 11 10:43 AD3.pdf


In [ ]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, TextStreamer, pipeline
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)


In [11]:
model.eval()
print(f"Model loaded on {DEVICE}")

Model loaded on cuda:0


In [ ]:
#######  USING LANGCHAIN CHROMA DB ###############

In [12]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [13]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('.', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

PdfStreamError: ignored

In [14]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

NameError: ignored

In [15]:
###### HF embedings

In [ ]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer


In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [ ]:
##############  RETRIEVER ################

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("Reinforcement Learning Based Techniques?")

In [ ]:
docs[0]

In [ ]:
docs

In [ ]:
##### QA

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=model,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
#query = "What is Flash attention?"
llm_response = qa_chain("Give a summary of all the techniques")
process_llm_response(llm_response)

In [ ]:
# import
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
#from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import chromadb


In [ ]:
# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [ ]:
documents = SimpleDirectoryReader(
    input_files=["AD1.pdf","AD2.pdf","AD3.pdf"]
).load_data()

# CHROMA-DB

In [ ]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
#chroma_client.delete_collection("AD_paper")
chroma_collection = chroma_client.create_collection("AD_paper")

### Set up ChromaVectorStore and load in data


In [ ]:
# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
service_context = ServiceContext.from_defaults(embed_model=embed_model)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context,
                                        service_context=service_context)

In [ ]:
index

In [ ]:
# Query Data
#query_engine = index.as_query_engine()

#response = query_engine.query("Reinforcement Learning Based Techniques?")

#display(Markdown(f"{response}"))

In [ ]:
#response = query_engine.query("Give a summary of all the techniques?")

#display(Markdown(f"{response}"))

### How to Persist: Saving to Disk

In [ ]:
db = chromadb.PersistentClient(path="./chroma_db")


chroma_collection = db.get_or_create_collection("AD_paper")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

service_context = ServiceContext.from_defaults(embed_model=embed_model,
                                               chunk_size=800,
                                               chunk_overlap=20)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

### Load from Disk

In [ ]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")

chroma_collection = db2.get_or_create_collection("AD_paper")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [ ]:
# Query Data
#query_engine = index.as_query_engine()

#response = query_engine.query("Reinforcement Learning Based Techniques?")

#display(Markdown(f"{response}"))

In [ ]:
#index.describe_index_stats()

###  **3 using Local model instead of open API**


In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


from IPython.display import Markdown, display

In [ ]:

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:

generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:

res = generate_text("Explain to me the difference between nuclear fission and fusion.")
print(res[0]["generated_text"])

Explain to me the difference between nuclear fission and fusion.

Nuclear fission is a process in which an atomic nucleus splits into two or more smaller nuclei, releasing a large amount of energy in the process. This process typically occurs when an atom is bombarded with a high-energy particle, such as a neutron. When the nucleus splits, it releases a large amount of energy in the form of kinetic energy of the fragments and gamma radiation.

Nuclear fusion, on the other hand, is the process by which two or more atomic nuclei combine to form a single, heavier nucleus. This process also releases a large amount of energy, but it does so at much higher temperatures than those required for fission. In order to achieve fusion, the atoms must be heated to incredibly high temperatures, typically over 100 million degrees Celsius.

One key difference between fission and fusion is the direction of the energy release. In fission, the energy is released outward from the nucleus, while in fusion, 

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context,
)

service_context = ServiceContext.from_defaults(
    llm=model, embed_model=embed_model
)
#set_global_service_context(service_context)

service_context = ServiceContext.from_defaults(embed_model=embed_model,
                                               chunk_size=800,
                                               chunk_overlap=20)

# And set the service context
set_global_service_context(service_context)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)


In [ ]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("Reinforcement Learning Based Techniques?")

display(Markdown(f"{response}"))